In [18]:
import pandas as pd
import numpy as np
import ast
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pickle

movies = pd.read_csv("tmdb_5000_movies.csv")
credits = pd.read_csv("tmdb_5000_credits.csv")

movies = movies.merge(credits, on='title')

movies = movies[['movie_id','title','overview','genres','keywords','cast','crew']]
movies.dropna(inplace=True)

def convert(text):
    return [i['name'] for i in ast.literal_eval(text)]

movies['genres'] = movies['genres'].apply(convert)
movies['keywords'] = movies['keywords'].apply(convert)

def convert_cast(text):
    return [i['name'] for i in ast.literal_eval(text)[:3]]

movies['cast'] = movies['cast'].apply(convert_cast)

def fetch_director(text):
    for i in ast.literal_eval(text):
        if i['job'] == 'Director':
            return [i['name']]
    return []

movies['crew'] = movies['crew'].apply(fetch_director)

movies['overview'] = movies['overview'].apply(lambda x: x.split())

for col in ['genres','keywords','cast','crew']:
    movies[col] = movies[col].apply(lambda x: [i.replace(" ","") for i in x])

movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']
movies['tags'] = movies['tags'].apply(lambda x: " ".join(x))

new_df = movies[['movie_id','title','tags']]
new_df['tags'] = new_df['tags'].apply(lambda x: x.lower())

cv = CountVectorizer(max_features=5000, stop_words='english')
vectors = cv.fit_transform(new_df['tags']).toarray()

similarity = cosine_similarity(vectors)

pickle.dump(new_df, open('movies.pkl','wb'))
pickle.dump(similarity, open('similarity.pkl','wb'))


/tmp/ipython-input-2693114231.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x: x.lower())


In [19]:
# import streamlit as st
# import pickle
# import pandas as pd

# movies = pickle.load(open('movies.pkl','rb'))
# similarity = pickle.load(open('similarity.pkl','rb'))

# def recommend(movie):
#     index = movies[movies['title'] == movie].index[0]
#     distances = similarity[index]
#     movie_list = sorted(
#         list(enumerate(distances)),
#         reverse=True,
#         key=lambda x: x[1]
#     )[1:6]

#     return [movies.iloc[i[0]].title for i in movie_list]

# st.title("🎬 Movie Recommendation System")

# selected_movie = st.selectbox(
#     "Select a movie",
#     movies['title'].values
# )

# if st.button("Recommend"):
#     for movie in recommend(selected_movie):
#         st.write("👉", movie)


In [20]:
!pip install streamlit


In [21]:
cv = CountVectorizer(max_features=3000, stop_words='english')


In [22]:
pickle.dump(new_df, open('movies.pkl','wb'))
pickle.dump(similarity, open('similarity.pkl','wb'))


In [23]:
%%writefile .gitignore
similarity.pkl
movies.pkl


Overwriting .gitignore


In [24]:
!ls -a


.   .config	.ipynb_checkpoints  sample_data     tmdb_5000_credits.csv
..  .gitignore	movies.pkl	    similarity.pkl  tmdb_5000_movies.csv


In [25]:
from google.colab import files
files.download('.gitignore')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>